In [120]:
import os
import tweepy
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import re


In [121]:
load_dotenv()
api_key = os.environ["TWITTER_API_KEY"]
api_secret = os.environ["TWITTER_API_SECRET"]
access_token = os.environ["TWITTER_API_ACCESS_TOKEN"]
access_token_secret = os.environ["TWITTER_API_ACCESS_TOKEN_SECRET"]
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [122]:
statuses = api.home_timeline(count=200)

In [123]:
statuses_json_list = list(map(lambda x: x._json, statuses))

status_df = pd.json_normalize(statuses_json_list)

a = status_df.created_at[0]

In [124]:
status_df.to_csv("twitter_full_status.csv") #大量に情報があるので今後の参考にどうぞ

In [125]:
#DataFrame内のドットをアンダースコアに置き換え
status_df.columns = [str(s).replace(' ', '_').replace('.', '_') for s in status_df.columns]
#画像か動画が入ってるツイートを抽出
image_df = status_df.query('entities_media  == entities_media')[['id_str','entities_media']] 

In [126]:
pattern = 'RT' # ここに禁則文字の正規表現を書く（hoge|fuga）とかかくとhogeとfugaが入ったツイートが出てくる
re_pattern = re.compile(pattern)
text_temp_df = status_df[status_df['text'].str.contains(re_pattern)]
text_df = text_temp_df[['id_str','text']]
text_df.to_csv("怪しいツイート.csv") #まあ名前は何でもいいんですケド

In [129]:
#こちらからフォローしているユーザーIDを全件取得
follow_ids = []
# フォローした人のIDを全部取得
for friend_id in api.get_friend_ids(screen_name="@Diphyl_Grayi"):
    follow_ids.append(friend_id)

user_df = pd.DataFrame( columns=['id','screen_name', 'user_name'] )
# 100IDsずつに詳細取得
for i in range(0, len(follow_ids), 100):
    for i in range(0,len(follow_ids)):
        #print(follow_ids[i])
        for user in api.lookup_users(user_id=[follow_ids[i]]):
            print(user.id_str + " " + user.screen_name + " " + user.name)
            temp_series = pd.Series([user.id_str, user.screen_name, user.name], name=i) 
            print(temp_series)
            user_df.append(temp_series)
            break
user_df.to_csv("user.csv")

1430941594023186437 kotori_fanza ことり〜小鳥〜🐤
0    1430941594023186437
1           kotori_fanza
2               ことり〜小鳥〜🐤
Name: 0, dtype: object
1110042293145235456 h_matsuda0 ガバガバサラリマン
0    1110042293145235456
1             h_matsuda0
2              ガバガバサラリマン
Name: 1, dtype: object
1461905674284068868 _koharu__fanza 白雪こはる🌸FANZAバーチャル
0    1461905674284068868
1         _koharu__fanza
2       白雪こはる🌸FANZAバーチャル
Name: 2, dtype: object
1426133222681833472 Daisukidayo_Nue 🎼🐈‍⬛＊ 鵺 ＊🥟/🐰🎀/🦖/🐈🍡/🌼❁/🌈🦄/🐺🌕/🍑💗🍑
0                1426133222681833472
1                    Daisukidayo_Nue
2    🎼🐈‍⬛＊ 鵺 ＊🥟/🐰🎀/🦖/🐈🍡/🌼❁/🌈🦄/🐺🌕/🍑💗🍑
Name: 3, dtype: object
1460946933544288258 2525chat_lov 桃甘 つるぎ🎀🍼
0    1460946933544288258
1           2525chat_lov
2               桃甘 つるぎ🎀🍼
Name: 4, dtype: object
102078646 imayaranakutemo T.Uchida
0          102078646
1    imayaranakutemo
2           T.Uchida
Name: 5, dtype: object
2981440862 silence_luvlive Kaz@あや
0         2981440862
1    silence_luvlive
2             Kaz@あや
Name: 6, dt

KeyboardInterrupt: 